In [ ]:
#!pip install simple_salesforce #allows us to work with salesforce API

In [1]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\ETL_Project'

In [2]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

## Import Dependencies

In [3]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [4]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [5]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [6]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [ ]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head(5)

## Transform Data

In [ ]:
#student_data['FullName'] = student_data['FirstName'].str.cat(student_data['LastName'], sep=" ")
student_data=student_data.rename(columns={"ID_Student":"ID_Student__c","StudentID":"StudentID__c","LastName":"Name",
                                       "FirstName":"FirstName__c","MiddleName":"MiddleName__c","Gender":"Gender__c",
                                        "BirthDate":"BirthDate__c"})
student_data.head(5)

## Convert DataFrame to a list of Dictionaries

In [ ]:
student_load = student_data.to_dict('records')
student_load

### Inserting rows in bulk

In [ ]:
try:
    sf.bulk.Student__c.insert(student_load) #Student__c is the API name
except Exception as e:
    print(e)

## Class Participant 

### Query Data

In [15]:
participant_data = pd.read_sql("SELECT * FROM classparticipant", conn)
participant_data.head(30)

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None
5,6,37,1,2020-03-16,None
6,7,38,1,2020-03-16,None
7,8,39,1,2020-03-16,None
8,9,40,1,2020-03-16,None
9,10,41,1,2020-03-16,None


### Rename Columns

In [16]:
participant_data=participant_data.rename(columns={"ID_ClassParticipant":"Name","ID_Student":"ID_Student__c",
                                        "ID_Class":"ID_Class__c","StartDate":"StartDate__c","EndDate":"EndDate__c"})
#convert date to string otherwise we will get JSON nonserilizable error
participant_data["StartDate__c"]=participant_data["StartDate__c"].astype(str)
participant_data.head()

,Name,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


### Extract Record ID 

In [17]:
query=sf.query("SELECT ID_Student__c from Student__c")

In [18]:
query

OrderedDict([('totalSize', 30),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0A3h000002c6yWEAQ')])),
                            ('ID_Student__c', '33')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0A3h000002c6yXEAQ')])),
                            ('ID_Student__c', '34')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0A3h000002c6yYEAQ')])

### Extract Student table record numbers

In [19]:
student_dict={}
x=0
while x< len(query['records']):
    url=query['records'][x]['attributes']['url'].split('/')[6]
    student_id=int(query['records'][x]['ID_Student__c'])
    print(url,student_id)
    student_dict[student_id]=url
    x=x+1
print(student_dict)

a0A3h000002c6yWEAQ 33
a0A3h000002c6yXEAQ 34
a0A3h000002c6yYEAQ 35
a0A3h000002c6yZEAQ 36
a0A3h000002c6yaEAA 37
a0A3h000002c6ybEAA 38
a0A3h000002c6ycEAA 39
a0A3h000002c6ydEAA 40
a0A3h000002c6yeEAA 41
a0A3h000002c6yfEAA 42
a0A3h000002c6ygEAA 43
a0A3h000002c6yhEAA 44
a0A3h000002c6yiEAA 45
a0A3h000002c6yjEAA 46
a0A3h000002c6ykEAA 47
a0A3h000002c6ylEAA 48
a0A3h000002c6ymEAA 49
a0A3h000002c6ynEAA 50
a0A3h000002c6yoEAA 51
a0A3h000002c6ypEAA 52
a0A3h000002c6yqEAA 53
a0A3h000002c6yrEAA 54
a0A3h000002c6ysEAA 55
a0A3h000002c6ytEAA 56
a0A3h000002c6yuEAA 57
a0A3h000002c6yvEAA 58
a0A3h000002c6ywEAA 59
a0A3h000002c6yxEAA 60
a0A3h000002c6yyEAA 61
a0A3h000002c6yzEAA 62
{33: 'a0A3h000002c6yWEAQ', 34: 'a0A3h000002c6yXEAQ', 35: 'a0A3h000002c6yYEAQ', 36: 'a0A3h000002c6yZEAQ', 37: 'a0A3h000002c6yaEAA', 38: 'a0A3h000002c6ybEAA', 39: 'a0A3h000002c6ycEAA', 40: 'a0A3h000002c6ydEAA', 41: 'a0A3h000002c6yeEAA', 42: 'a0A3h000002c6yfEAA', 43: 'a0A3h000002c6ygEAA', 44: 'a0A3h000002c6yhEAA', 45: 'a0A3h000002c6yiEAA', 4

In [20]:
for index,row in participant_data.iterrows():
    participant_data['ID_Student__c'][index]=student_dict[row['ID_Student__c']]
participant_data

C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c
0,1,a0A3h000002c6yWEAQ,1,2020-03-16,None
1,2,a0A3h000002c6yXEAQ,1,2020-03-16,None
2,3,a0A3h000002c6yYEAQ,1,2020-03-16,None
3,4,a0A3h000002c6yzEAA,1,2020-03-16,None
4,5,a0A3h000002c6yZEAQ,1,2020-03-16,None
5,6,a0A3h000002c6yaEAA,1,2020-03-16,None
6,7,a0A3h000002c6ybEAA,1,2020-03-16,None
7,8,a0A3h000002c6ycEAA,1,2020-03-16,None
8,9,a0A3h000002c6ydEAA,1,2020-03-16,None
9,10,a0A3h000002c6yeEAA,1,2020-03-16,None


### Convert Dataframe to list of dictionaries

In [21]:
participant_load = participant_data.to_dict('records')
participant_load

[{'Name': 1,
  'ID_Student__c': 'a0A3h000002c6yWEAQ',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 2,
  'ID_Student__c': 'a0A3h000002c6yXEAQ',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 3,
  'ID_Student__c': 'a0A3h000002c6yYEAQ',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 4,
  'ID_Student__c': 'a0A3h000002c6yzEAA',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 5,
  'ID_Student__c': 'a0A3h000002c6yZEAQ',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 6,
  'ID_Student__c': 'a0A3h000002c6yaEAA',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 7,
  'ID_Student__c': 'a0A3h000002c6ybEAA',
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'Name': 8,
  'ID_Student__c': 'a0A3h000002c6ycEAA',
  'ID_Class__c': 1,
  'StartDate__c': '2020

### Inserting rows in bulk

In [22]:
try:
    sf.bulk.ClassParticipant__c.insert(participant_load)
except Exception as e:
    print(e)

# Course

In [ ]:
course_data = pd.read_sql("SELECT * FROM course", conn)
course_data.head(30)

In [ ]:
course_data=course_data.rename(columns={"ID_Course":"ID_Course__c","CourseCode":"CourseCode__c","CourseName":"Name",
                                       "CreditHours":"CreditHours__c","BootCampCourse":"BootCampCourse__c"})
course_data.head()

In [ ]:
course_load = course_data.to_dict('records')
course_load

In [ ]:
try:
    sf.bulk.Course__c.insert(course_load)
except Exception as e:
    print(e)

# Class

In [32]:
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data.head(5)

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [33]:
class_data=class_data.rename(columns={"ID_Class":"ID_Class__c","ID_Course":"ID_Course__c","Section":"Name",
                                      "StartDate":"StartDate__c","EndDate":"EndDate__c"})

class_data.head(5)

,ID_Class__c,ID_Course__c,Name,StartDate__c,EndDate__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [34]:
class_data["StartDate__c"]=class_data["StartDate__c"].astype(str)
class_data["EndDate__c"]=class_data["EndDate__c"].astype(str)

In [35]:
course_query=sf.query("SELECT ID_Course__c from Course__c")

course_query

OrderedDict([('totalSize', 8),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a013h00000727q4AAA')])),
                            ('ID_Course__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a013h00000727q5AAA')])),
                            ('ID_Course__c', 2.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a013h00000727q6AAA')])),
        

In [36]:
course_dict={}
x=0
while x< len(course_query['records']):
    url=course_query['records'][x]['attributes']['url'].split('/')[6]
    course_id=int(course_query['records'][x]['ID_Course__c'])
    print(url,course_id)
    course_dict[course_id]=url
    x=x+1
print(course_dict)

a013h00000727q4AAA 1
a013h00000727q5AAA 2
a013h00000727q6AAA 3
a013h00000727q7AAA 4
a013h00000727q8AAA 5
a013h00000727q9AAA 6
a013h00000727qAAAQ 7
a013h00000727qBAAQ 8
{1: 'a013h00000727q4AAA', 2: 'a013h00000727q5AAA', 3: 'a013h00000727q6AAA', 4: 'a013h00000727q7AAA', 5: 'a013h00000727q8AAA', 6: 'a013h00000727q9AAA', 7: 'a013h00000727qAAAQ', 8: 'a013h00000727qBAAQ'}


In [37]:
for index,row in class_data.iterrows():
    class_data['ID_Course__c'][index]=course_dict[row['ID_Course__c']]
class_data

C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_Class__c,ID_Course__c,Name,StartDate__c,EndDate__c
0,1,a013h00000727q4AAA,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,a013h00000727q4AAA,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,a013h00000727q5AAA,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,a013h00000727q6AAA,GWARL201905UIUX3,2019-05-14,2019-11-07


In [38]:

#class_df=pd.merge(class_data,participant_data, on="ID_Class__c")


participant_query=sf.query("SELECT ID_Class__c from ClassParticipant__c")

participant_query


OrderedDict([('totalSize', 28),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'ClassParticipant__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/ClassParticipant__c/a0E3h000000GmZgEAK')])),
                            ('ID_Class__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'ClassParticipant__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/ClassParticipant__c/a0E3h000000GmZhEAK')])),
                            ('ID_Class__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'ClassParticipant__c'),
                                          ('url',
                                           '/services/data/v42.0/sob

In [39]:
#class_data=class_data[["ID_Class__c","ID_Course__c","Name","StartDate__c","EndDate__c","ID_Student__c"]]

In [40]:
participant_dict={}
x=0
while x< len(participant_query['records']):
    url=participant_query['records'][x]['attributes']['url'].split('/')[6]
    participant_id=int(participant_query['records'][x]['ID_Class__c'])
    print(url,participant_id)
    participant_dict[participant_id]=url
    x=x+1
print(participant_dict)

a0E3h000000GmZgEAK 1
a0E3h000000GmZhEAK 1
a0E3h000000GmZiEAK 1
a0E3h000000GmZjEAK 1
a0E3h000000GmZkEAK 1
a0E3h000000GmZlEAK 1
a0E3h000000GmZmEAK 1
a0E3h000000GmZnEAK 1
a0E3h000000GmZoEAK 1
a0E3h000000GmZpEAK 1
a0E3h000000GmZqEAK 1
a0E3h000000GmZrEAK 1
a0E3h000000GmZsEAK 1
a0E3h000000GmZtEAK 1
a0E3h000000GmZuEAK 1
a0E3h000000GmZvEAK 1
a0E3h000000GmZwEAK 1
a0E3h000000GmZxEAK 1
a0E3h000000GmZyEAK 1
a0E3h000000GmZzEAK 1
a0E3h000000Gma0EAC 1
a0E3h000000Gma1EAC 1
a0E3h000000Gma2EAC 1
a0E3h000000Gma3EAC 1
a0E3h000000Gma4EAC 1
a0E3h000000Gma5EAC 1
a0E3h000000Gma6EAC 1
a0E3h000000Gma7EAC 1
{1: 'a0E3h000000Gma7EAC'}


In [41]:
for index,row in class_data.iterrows():
    try:
        class_data['ID_Class__c'][index]=participant_dict[row['ID_Class__c']]
    except KeyError as e:
        print(e)
class_data

2
3
4


C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID_Class__c,ID_Course__c,Name,StartDate__c,EndDate__c
0,a0E3h000000Gma7EAC,a013h00000727q4AAA,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,a013h00000727q4AAA,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,a013h00000727q5AAA,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,a013h00000727q6AAA,GWARL201905UIUX3,2019-05-14,2019-11-07


In [42]:
class_load = class_data.to_dict('records')
class_load

[{'ID_Class__c': 'a0E3h000000Gma7EAC',
  'ID_Course__c': 'a013h00000727q4AAA',
  'Name': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': '2020-03-16',
  'EndDate__c': '2020-03-14'},
 {'ID_Class__c': 2,
  'ID_Course__c': 'a013h00000727q4AAA',
  'Name': 'GWDC201805DATA3',
  'StartDate__c': '2018-05-15',
  'EndDate__c': '2018-11-08'},
 {'ID_Class__c': 3,
  'ID_Course__c': 'a013h00000727q5AAA',
  'Name': 'GWARL201905WEB3',
  'StartDate__c': '2019-05-14',
  'EndDate__c': '2019-11-07'},
 {'ID_Class__c': 4,
  'ID_Course__c': 'a013h00000727q6AAA',
  'Name': 'GWARL201905UIUX3',
  'StartDate__c': '2019-05-14',
  'EndDate__c': '2019-11-07'}]

In [43]:
try:
    sf.bulk.Class__c.insert(class_load)
except Exception as e:
    print(e)

In [ ]:
sf.ClassParticipant.update('1',{'ID_Class': 'a0E3h000000Gma7EAC'})